# ECMB paleogeographic reconstructions generation

This Jupyter notebook develops paleogeographic reconstructions for Laurentia using the ```pygplates``` module (it therefore needs to be run with a Python 2.7 kernel). 

This Jupyter notebook accompanies the paper:

Swanson-Hysell et al. 2021 **The paleogeography of Laurentia in its early years: new constraints from the Paleoproterozoic East Central Minnesota batholith**

## Terrane plate IDs

| Tectonic Block | Plate ID | Plate ID note |
|------|-----|-----|
| Churchill Province | 10496 | corresponds to Churchill (SE) in EE |
| Hearne Province | 10015 | corresponds to Hearne South in EE |
| Greenland_Aasiaat | 10028 | corresponds to Aasiaat in EE |
| Greenland_Inglefield | 10568 | corresponds to Inglefield in EE |
| Greenland_Ketilidian | 10018 | corresponds to Ketilidian in EE |
| Greenland_NorthAtlantic | 10032 | corresponds to North Atlantic in EE |
| Greenland_Rae | 10036 | corresponds to Rae (Greenland) in EE |
| Grouse Creek Block | 19049 |  No equivelant block in EE (grouped with Wyoming) |
| Marshfield Terrane | 19000 | No equivelant block in EE |
| Medicine Hat Block | 10551 | corresponds to Medicine Hat in EE|
| MetaIncognita Block | 10545 | Not given this name in WK07, but has differential motion relative to Rae; corresponds to MetaIncognita Block in EE |
| Nain | 10029 | corresponds to Nain in EE (mainland Canada) |
| Mojavia | 10052 | corresponds to Mojave in EE |
| Nova | 10549 | Cordillera Archean Block in WK07; corresponds to Paleoproteroic Nova Block in EE |
| Rae Province | 10640 | corresponds to Rae (SW) in EE |
| Sask craton | 10039 | correspond to Sask in EE |
| Scotland | 10258 | not a block in the WK07 model, but assigned to pole list; corresponds to N Highlands in EE
| Slave Province | 10041 | corresponds to Slave W in EE |
| Superior (east) | 10043 | corresponds to Superior (Abitibi) in EE |
| Superior (west) | 10023 | corresponds to Superior (Minnesota River Valley) in EE |
| Svalbard | 10296 | corresponds to Svalbard E in EE |
| Wyoming Province | 10049 | corresponds to Wyoming in EE |

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pygplates

## Make GMPL files of poles

In [2]:
def make_GPML(pmag_data,output_file_name):
    vgps = []
    for n in pmag_data.index:
        # The pole position and the average sample site position.
        pole_position = pygplates.PointOnSphere(pmag_data.PLAT[n], pmag_data.PLONG[n])
        average_sample_site_position = pygplates.PointOnSphere(pmag_data.SLAT[n], pmag_data.SLONG[n])

        # Create the virtual geomagnetic pole feature.
        virtual_geomagnetic_pole_feature = pygplates.Feature.create_reconstructable_feature(
            pygplates.FeatureType.gpml_virtual_geomagnetic_pole,pole_position,
            name=str(pmag_data.ROCKNAME[n]),reconstruction_plate_id=int(pmag_data.Plate_ID[n]))
        
        #, reconstruction_plate_id=str(pmag_data.Plate_ID[n]))

        # Set the average sample site position.
        # We need to specify its property name otherwise it defaults to the pole position and overwrites it.
        virtual_geomagnetic_pole_feature.set_geometry(
            average_sample_site_position,
            pygplates.PropertyName.gpml_average_sample_site_position) 
        
        # Set the pole position uncertainty and the average age.
        virtual_geomagnetic_pole_feature.set_double(
            pygplates.PropertyName.gpml_pole_a95,
            float(pmag_data.A95[n]))
        virtual_geomagnetic_pole_feature.set_double(
            pygplates.PropertyName.gpml_average_age,
            pmag_data['nominal age'][n])
        vgps.append(virtual_geomagnetic_pole_feature)
        
    output_features = pygplates.FeatureCollection(vgps)

    # Note that the output format could be shp, gmt, gpml or gpmlz - the extension controls the format
    output_features.write(output_file_name)

In [3]:
Leirubakki_poles = pd.read_csv('../data/pole_compilations/Leirubakki_poles_w_PlateID.csv')
Leirubakki_poles.dropna(subset = ["Plate_ID"], inplace=True)
make_GPML(Leirubakki_poles,'../data/pole_compilations/Laurentia_poles.gpml')

## Make Laurentia reconstructions with a simple Laurentia rotation model

In [4]:
rotation_model = pygplates.FeatureCollection('../Data/pole_compilations/simple_Laurentia_model.rot')

anchor_plate = 0
Laurentia_pre_430 = '../Data/GIS/Laurentia.shp'

time_list = [2212,2110,2018,1888,1868,1760,0]

pygplates.reconstruct(Laurentia_pre_430, rotation_model, 'visualizations/reconstructions/0Ma_Laurentia_pre_430.shp', 0, anchor_plate)
#for reconstruction_time in [1740,1440,1140,740]:
for reconstruction_time in time_list:
    export_filename = 'visualizations/reconstructions/{0}Ma_Laurentia.shp'.format(reconstruction_time)
    pygplates.reconstruct(Laurentia_pre_430, rotation_model, export_filename, reconstruction_time, anchor_plate)

Calculate and export the Euler poles for Laurentia associated with the reconstruction times and export to .csv

In [5]:
def make_euler_file(plate_id,times,input_rotation_filename):
    
    empty_series = pd.Series(np.zeros(len(times)))
    rotations = pd.DataFrame(columns = ['time', 'pole_lat', 'pole_lon', 'pole_angle'])
    rotations['time'] = empty_series
    rotations['pole_lat'] = empty_series
    rotations['pole_lon'] = empty_series
    rotations['pole_angle'] = empty_series

    rotation_model_object = pygplates.RotationModel(input_rotation_filename)

    for n in range(0,len(times)):
        finite_rotation = rotation_model_object.get_rotation(times[n],plate_id,0,0)
        pole_lat,pole_lon,pole_angle = finite_rotation.get_lat_lon_euler_pole_and_angle_degrees()
        rotations['time'][n] = times[n]
        rotations['pole_lat'][n] = pole_lat
        rotations['pole_lon'][n] = pole_lon
        rotations['pole_angle'][n] = pole_angle

    rotations.to_csv('visualizations/reconstructions/euler_rotations_{0}.csv'.format(plate_id))
    
make_euler_file(10043,[2212,2110,2018,1888,1868,1760,0],'../Data/pole_compilations/simple_Laurentia_model.rot')
make_euler_file(10023,[2212,2110,2018,1888,1868,1760,0],'../Data/pole_compilations/simple_Laurentia_model.rot')
make_euler_file(10041,[2212,2110,2018,1888,1868,1760,0],'../Data/pole_compilations/simple_Laurentia_model.rot')
make_euler_file(10640,[2212,2110,2018,1888,1868,1760,0],'../Data/pole_compilations/simple_Laurentia_model.rot')
make_euler_file(1000,[2212,2110,2018,1888,1868,1760,0],'../Data/pole_compilations/simple_Laurentia_model.rot')